In [ ]:
import os
import tarfile
from six.moves import urllib

In [ ]:
download_root="https://raw.githubusercontent.com/ageron/handson-ml2/master/"
housing_path=os.path.join("datasets","housing")
#print(os.path.join("datasets","housing"))
housing_url=download_root+"datasets/housing/housing.tgz"

def fetch_housing_data(housing_url,housing_path):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    #print((os.path.isdir(housing_path)))
    tgz_path=os.path.join(housing_path,"housing.tgz")
    urllib.request.urlretrieve(housing_url,tgz_path)
    housing_tgz=tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
fetch_housing_data(housing_url,housing_path)


In [ ]:
import pandas as pd
hdf=pd.read_csv(housing_path+"\housing.csv")
hdf.head(5)


In [ ]:
hdf.info()

In [ ]:
hdf['ocean_proximity'].value_counts()

In [ ]:
(hdf.describe())

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
hdf.hist(bins=50, figsize=(20,15))
#hdf.hist()
plt.show()

In [ ]:
#creating test data set
#This is tradiationalway but not  preffered as the indicies are generated at random
import numpy as np
# def train_test_data(hdf,test_ratio):
#     shuffle_indicies=np.random.permutation(len(hdf))
#     test_size=int(len(hdf)*test_ratio)
#     test_data=hdf.iloc[shuffle_indicies[:test_size]]
#     train_data=hdf.iloc[shuffle_indicies[test_size:]]
#     return test_data,train_data

# test_data,train_data=train_test_data(hdf,0.2)
# print(len(train_data))
# print(len(test_data))

from sklearn.model_selection import train_test_split
train_set,test_set= train_test_split(hdf, test_size=0.2, random_state=42)

In [ ]:
hdf["income_cat"]=pd.cut(hdf['median_income'],bins=[0.,1.5,3,4.5,6.,np.inf],labels=[1,2,3,4,5])
hdf["income_cat"].hist()

In [ ]:
#creating statifide samples
from sklearn.model_selection import StratifiedShuffleSplit
split=StratifiedShuffleSplit(n_splits=1,test_size=0.2, random_state=42)
for train_index, test_index in split.split(hdf, hdf["income_cat"]):
    strat_train_set=hdf.loc[train_index]
    strat_test_set=hdf.loc[test_index]
print(strat_test_set["income_cat"].value_counts()/len(strat_test_set))
    

In [ ]:
#dropping income_cat attribute from hdf
for set_ in (strat_train_set,strat_test_set):
    #print((strat_train_set.head(5),strat_test_set.head(5)))
    set_.drop("income_cat",axis=1,inplace=True)
    
    

In [ ]:
train_dup=strat_train_set.copy()
train_dup.plot(kind="scatter",x="longitude",y="latitude",alpha=0.2)

In [ ]:
#calculating corr between two attributes
corr_matrix=train_dup.corr()
#print(corr_matrix)
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
train_dup.plot(kind="scatter",x="longitude",y="latitude",alpha=0.4,s=train_dup["population"]/100,label="population",
              figsize=(10,7), c="median_house_value",cmap=plt.get_cmap("jet"),colorbar=True,sharex=False)
plt.legend()

In [ ]:
#ploting scatter_matix for correation 
# import pandas.plotting
from pandas.plotting import scatter_matrix
attributes=["median_house_value","median_income","total_rooms","housing_median_age","latitude"]
scatter_matrix(train_dup[attributes],figsize=(12,8))

In [ ]:
train_dup.plot(kind="scatter", x="median_income",y="median_house_value",alpha=0.1)
plt.show()

In [ ]:
#checking other attributes contibution
train_dup["rooms_per_household"]=train_dup["total_rooms"]/train_dup["households"]
train_dup["bedrooms_per_rooms"]=train_dup["total_bedrooms"]/train_dup["total_rooms"]
train_dup["population_per_household"]=train_dup["population"]/train_dup["households"]
corr_matrix=train_dup.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

Data Cleaning

In [26]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy="median")
#droping text attributes
train_num=strat_train_set.drop("ocean_proximity",axis=1)
imputer.fit(train_num)
imputer.statistics_
X=imputer.transform(train_num)
train_df=pd.DataFrame(X,columns=train_num.columns)